In [ ]:
from os import path

import geopandas as gpd
import osmnx as ox
import pandas as pd
import requests
import tqdm
from owslib.wfs import WebFeatureService

from swiss_urban_trees import geo_utils

In [ ]:
# ICA WFS
wfs_url = (
    "https://app2.ge.ch/tergeoservices/services/Hosted/"
    "SIPV_ICA_ARBRE_ISOLE/MapServer/WFSServer"
)
wfs_version = "2.0.0"
wfs_api_crs = "epsg:2056"
wfs_data_crs = "epsg:2056"
tile_size = 200  # in meters

aoi = "Geneva, Switzerland"
dst_filepath = "../data/raw/sitg-orthophoto-2019-tiles/ica-trees.gpkg"

In [ ]:
if path.exists(aoi):
    aoi_gdf = gpd.read_file(aoi)
else:
    aoi_gdf = ox.geocode_to_gdf(aoi)
aoi_gdf = aoi_gdf.to_crs(wfs_api_crs)

In [ ]:
region_tile_gser = geo_utils.get_tile_gser(
    aoi_gdf.geometry.iloc[0],
    tile_size,
    tile_size,
    wfs_api_crs,
)

In [ ]:
wfs = WebFeatureService(wfs_url)
# there should be only one layer
layer_name = list(wfs.contents)[0]
wfs_params_base = dict(
    service="WFS",
    version=wfs_version,
    request="GetFeature",
    # typeName="SIPV_ICA_ARBRE_ISOLE",
    typeName=layer_name,
    # outputFormat="GeoJSON",
    srsname=wfs_api_crs,
)
wfs_gdf = gpd.GeoDataFrame()
for region_window in tqdm.tqdm(region_tile_gser):
    wfs_params = wfs_params_base.copy()
    wfs_params["BBOX"] = ",".join(map(str, region_window.bounds))
    query_url = requests.Request("GET", wfs_url, params=wfs_params).prepare().url
    try:
        wfs_gdf = pd.concat([wfs_gdf, gpd.read_file(query_url)], axis="rows")
    except IndexError:
        # assume no data in this window
        pass
    except Exception as e:
        print(f"Failed reading data for {query_url}, exception: {e}")

In [ ]:
wfs_gdf.to_file(dst_filepath)